In [64]:
import pandas as pd
import numpy as np
import requests
import time

The dataset selected initially is "Air Quality Annual Summary" from ...

In [ ]:
!kaggle datasets download -d epa/air-quality

In [5]:
data_historic = pd.read_csv("C:\\Users\\Victo\\Downloads\\epa_air_quality_annual_summary.csv")
data_historic.head(5)

C:\Users\Victo\AppData\Local\Temp\ipykernel_4552\565861268.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_historic = pd.read_csv("C:\\Users\\Victo\\Downloads\\epa_air_quality_annual_summary.csv")


,state_code,county_code,site_num,parameter_code,poc,latitude,longitude,datum,parameter_name,sample_duration,...,seventy_five_percentile,fifty_percentile,ten_percentile,local_site_name,address,state_name,county_name,city_name,cbsa_name,date_of_last_change
0,20,57,1,82134,1,37.771964,-100.018190,WGS84,Molybdenum PM10 STP,24 HOUR,...,0.00000,0.0,0.0,DODGE CITY,PUMP STATION 2100 1ST AVE,Kansas,Ford,Dodge City,Dodge City KS,2012-08-11
1,18,163,12,42401,1,38.021727,-87.569458,WGS84,Sulfur dioxide,3-HR BLK AVG,...,3.60000,1.3,0.0,Evansville CAAP,425 WEST MILL ROAD/ FIRE STATION #17,Indiana,Vanderburgh,Evansville,Evansville IN-KY,2013-02-16
2,40,31,9000,88152,1,34.732404,-98.713351,WGS84,Phosphorus PM2.5 LC,24 HOUR,...,0.00000,0.0,0.0,NaN,Wichita Mountains,Oklahoma,Comanche,NaN,Lawton OK,2016-02-26
3,32,3,561,42601,1,36.163959,-115.113916,WGS84,Nitric oxide (NO),1 HOUR,...,18.80000,0.9,0.0,Sunrise Acres,2501 SUNRISE AVENUE,Nevada,Clark,Las Vegas,Las Vegas-Henderson-Paradise NV,2015-03-20
4,13,127,3001,12154,1,31.185187,-81.485166,WGS84,Selenium (TSP) STP,24 HOUR,...,0.00064,0.0,0.0,NaN,BRUNSWICK COASTAL COLLEGE,Georgia,Glynn,Brunswick,Brunswick GA,2015-07-22


In [7]:
data_historic.shape

(2038710, 55)

In [49]:
parameter_names = list(data_historic.parameter_name.unique())
print(len(parameter_names), parameter_names)

1134 ['Molybdenum PM10 STP', 'Sulfur dioxide', 'Phosphorus PM2.5 LC', 'Nitric oxide (NO)', 'Selenium (TSP) STP', 'Black Carbon PM2.5 at 880 nm', 'Phosphorus PM10-2.5 STP', 'Coarse Mass Extinction PM2.5 LC', 'Benzene', 'Ethylene dichloride', 'Elemental Carbon Extinction PM2.5 LC', 'Carbon monoxide', 'Manganese (TSP) STP', 'Cadmium (TSP) STP', 'Ozone', 'Ethylene dibromide', 'PM2.5 - Local Conditions', 'OC1 PM2.5 LC', 'Calcium PM2.5 STP', '123-Trimethylbenzene', 'Tin PM2.5 LC', 'PM10 Total 0-10um STP', 'Sample Flow Rate- CV', 'Manganese PM10 STP', 'Sample Min Baro Pressure', 'n-Pentane', 'Indium PM2.5 LC', '13-Butadiene', 'cis-2-Pentene', 'Ethylbenzene', 'Tungsten PM2.5 LC', 'Methyl tert-butyl ether', 'Average Ambient Temperature', 'Manganese PM2.5 LC', 'Acetylene', 'Ethyl acrylate', 'Molybdenum (TSP) STP', 'Carbon disulfide', 'Arsenic (TSP) STP', 'Wind Speed - Resultant', 'Sample Max Baro Pressure', 'Methyl Butyl Ketone', '3-Methyl-1-butene', 'OC CSN_Rev Unadjusted PM2.5 LC TOR', 'Total 

In [55]:
filtered_historic = data_historic[data_historic['parameter_code'] == 45201] 
filtered_historic.head()

,state_code,county_code,site_num,parameter_code,poc,latitude,longitude,datum,parameter_name,sample_duration,...,seventy_five_percentile,fifty_percentile,ten_percentile,local_site_name,address,state_name,county_name,city_name,cbsa_name,date_of_last_change
8,42,101,4,45201,2,40.008889,-75.097780,WGS84,Benzene,24 HOUR,...,4.43,3.58,2.59,Air Management Services Laboratory (AMS LAB),1501 E. LYCOMING AVE.,Pennsylvania,Philadelphia,Philadelphia,Philadelphia-Camden-Wilmington PA-NJ-DE-MD,2015-07-22
129,48,355,32,45201,1,27.804505,-97.431582,WGS84,Benzene,24 HOUR,...,13.62,6.60,1.38,Corpus Christi Huisache,3810 Huisache Street,Texas,Nueces,Corpus Christi,Corpus Christi TX,2010-03-06
430,36,29,1007,45201,2,42.827300,-78.849840,WGS84,Benzene,24 HOUR,...,2.30,1.70,1.10,SIMON STREET,SIMON STREET AND RIDGE ROAD,New York,Erie,Lackawanna,Buffalo-Cheektowaga-Niagara Falls NY,2010-03-06
578,51,87,14,45201,1,37.556520,-77.400270,WGS84,Benzene,24 HOUR,...,2.13,1.58,0.90,MathScience Innovation Center,2401 HARTMAN STREET MATH & SCIENCE CTR,Virginia,Henrico,East Highland Park,Richmond VA,2010-03-06
830,06,41,1,45201,3,37.972310,-122.520004,WGS84,Benzene,24 HOUR,...,1.74,1.14,0.54,San Rafael,534 4TH ST.,California,Marin,San Rafael,San Francisco-Oakland-Hayward CA,2012-08-22


For possible machine learning tasks I decided to narrow down columns 

I selected



In [65]:
filtered_historic = data_historic[data_historic['parameter_name'] == 'Ozone'] 
params_list = ["12103", "12105", "12110", "12112", "12115", "12128", "12132", "12136", "14115", "43218", "43502", "43503", "43505",
         "43509", "43802", "43803", "43804", "43815", "43817", "43818", "43824", "43829", "43830", "43831", "43843", "43860",
         "45201", "82103", "82105", "82110", "82112", "82128","82132","82136","82142","88103","88105","88110","88112","88128",
         "88132", "88136"]
filtered_historic.shape

(166005, 55)

Now I would use Air Quality System (AQS) API to enhance...

In [81]:
# data_modern = data_historic.drop(data_historic.index)
data_modern = pd.DataFrame(columns=['state_code','county_code','site_number','parameter_code','poc','latitude','longitude', 'datum',
                                   'parameter', 'sample_duration_code', 'sample_duration', 'pollutant_standard', 'metric_used',
                                   'method', 'year', 'units_of_measure', 'event_type', 'observation_count', 'observation_percent',
                                   'validity_indicator', 'valid_day_count', 'required_day_count', 'exceptional_data_count',
                                   'null_observation_count', 'primary_exceedance_count', 'secondary_exceedance_count',
                                   'certification_indicator', 'arithmetic_mean', 'standard_deviation', 'first_max_value', 'first_max_datetime',
                                   'second_max_value', 'second_max_datetime', 'third_max_value', 'third_max_datetime', 'fourth_max_value',
                                   'fourth_max_datetime', 'first_max_nonoverlap_value', 'first_max_n_o_datetime', 'second_max_nonoverlap_value',
                                   'second_max_n_o_datetime', 'ninety_ninth_percentile', 'ninety_eighth_percentile', 'ninety_fifth_percentile',
                                   'ninetieth_percentile', 'seventy_fifth_percentile', 'fiftieth_percentile', 'tenth_percentile', 'local_site_name',
                                   'site_address', 'state', 'county', 'city', 'cbsa_code', 'cbsa', 'date_of_last_change'])


In [11]:
key = 'sandgoose35'
email = 'VictorPakholkov@outlook.com'
base_url = 'https://aqs.epa.gov/data/api/annualData/'

In [51]:
params = {
    'email': email,
    'key': key,
    'param': '12103',  # Ozone parameter
    'bdate': '20180101',  # year 2018
    'edate': '20181212',  # year 2018
    'state': '01',  # state 01
}


In [52]:
response = requests.get(f'{base_url}byState', params=params)
if response.status_code == 200:
    data = response.json()
    for data_entry in data['Data']:
        data_to_append = pd.DataFrame([data_entry], columns=list(data_modern.columns))
        data_modern = pd.concat([data_modern, data_to_append], ignore_index=True)
else:
    print(f"API request failed with status code {response.status_code}")
    print(response.text)

API request failed with status code 400
{
  "Header": [
    {
      "status": "Failed",
      "request_time": "2023-11-21T16:59:23.256-05:00",
      "url": "https://aqs.epa.gov/data/api/annualData/byState?email=VictorPakholkov%40outlook.com&key=sandgoose35&param=ALL&bdate=20180101&edate=20181212&state=01",
      "error": [
        "parameter code: ALL, requires numeric value."
      ]
    }
  ]
}


In [66]:
params_list = ["12103,12105,12110,12112,12115", "12128,12132,12136,14115,43218", "43502,43503,43505,43509,43802", 
          "43803,43804,43815,43817,43818", "43824,43829,43830,43831,43843", "43860,45201,82103,82105,82110", 
          "82112,82128,82132,82136,82142", "88103,88105,88110,88112,88128", "88132,88136"]
bdates = ['20180101', '20190101', '20200101', '20210101', '20220101']
edates = ['20181212', '20191212', '20201212', '20211212', '20221212']
states = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
         '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', 
          '40', '41', '42', '43', '44', '45', '46','47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '66', '72', '78',
         '80', 'CC']

In [88]:
pause_duration = 5.5

for param in params_list:
    for i in range(len(bdates)):
        for state in states:
            # Construct the parameters for the API request
            api_params = {
                'email': email,
                'key': key,
                'param': param,
                'bdate': bdates[i],
                'edate': edates[i],
                'state': state
            }

            response = requests.get(f'{base_url}byState', params=api_params)
            # time.sleep(pause_duration)
            if response.status_code == 200:
                data = response.json()
                print(api_params)
                for data_entry in data['Data']:
                    data_to_append = pd.DataFrame([data_entry], columns=list(data_modern.columns))
                    data_modern = pd.concat([data_modern, data_to_append], ignore_index=True)
            else:
                print(f"Request failed for params: {api_params}")
                print(f"Status code: {response.status_code}")
                print(response.text)




{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '01'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '02'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '03'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '04'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '05'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '06'}
{'email': 'VictorPakholkov@o

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '52'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '53'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '54'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '55'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '56'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20180101', 'edate': '20181212', 'state': '66'}
{'email': 'VictorPakholkov@o

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '38'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '39'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '40'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '41'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '42'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20190101', 'edate': '20191212', 'state': '43'}
{'email': 'VictorPakholkov@o

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '24'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '25'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '26'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '27'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '28'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20200101', 'edate': '20201212', 'state': '29'}
{'email': 'VictorPakholkov@o

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '10'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '11'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '12'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '13'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '14'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': '15'}
{'email': 'VictorPakholkov@o

Request failed for params: {'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': 'CC'}
Status code: 400
{
  "Header": [
    {
      "status": "Failed",
      "request_time": "2023-11-21T18:34:30.907-05:00",
      "url": "https://aqs.epa.gov/data/api/annualData/byState?email=VictorPakholkov%40outlook.com&key=sandgoose35&param=12103%2C12105%2C12110%2C12112%2C12115&bdate=20210101&edate=20211212&state=CC",
      "error": [
        "state: CC, requires 2 digit numeric value."
      ]
    }
  ]
}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20210101', 'edate': '20211212', 'state': 'CC'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '01'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '47'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '48'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '49'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '50'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '51'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12103,12105,12110,12112,12115', 'bdate': '20220101', 'edate': '20221212', 'state': '52'}
{'email': 'VictorPakholkov@o

{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '33'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '34'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '35'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '36'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '37'}
{'email': 'VictorPakholkov@outlook.com', 'key': 'sandgoose35', 'param': '12128,12132,12136,14115,43218', 'bdate': '20180101', 'edate': '20181212', 'state': '38'}
{'email': 'VictorPakholkov@o

KeyboardInterrupt: 

In [89]:
print(data_modern.shape)
print(data_modern.head())

(3928, 56)
  state_code county_code site_number parameter_code poc   latitude  \
0         01         073        2003          12115   6  33.499722   
1         01         073        2003          12115   6  33.499722   
2         06         025        0005          12103   7  32.676180   
3         06         025        0005          12110   7  32.676180   
4         06         025        0005          12112   7  32.676180   

    longitude  datum              parameter sample_duration_code  ...  \
0  -86.924167  WGS84  Chromium VI (TSP) STP                    7  ...   
1  -86.924167  WGS84  Chromium VI (TSP) STP                    7  ...   
2 -115.483070  NAD83      Arsenic (TSP) STP                    7  ...   
3 -115.483070  NAD83      Cadmium (TSP) STP                    7  ...   
4 -115.483070  NAD83     Chromium (TSP) STP                    7  ...   

  fiftieth_percentile tenth_percentile        local_site_name  \
0             0.00023          0.00001                  Wylam   